# SUP ML 3 - PREDICT

# Librerias

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install catboost

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from catboost import CatBoostClassifier
import sklearn.metrics as metrics
import joblib
import pickle
pd.set_option('display.max_columns',None)

import warnings
warnings.filterwarnings("ignore")

# Carga modelo

* Cargar el modelo
* Obtener la lista de model features

In [ ]:
cboost = joblib.load('/content/drive/MyDrive/Master_Data_Science/DSC0324BCN-ENTREGABLE_2-Barria_Bau_Guimerà_Maninatt_Solieri/Modelo/cbc_model')
print('Modelo cargado')

Modelo cargado


In [ ]:
feature_names = cboost.feature_names_
print("Features cargadas:", feature_names)

Features cargadas: ['mou', 'totmrc', 'da', 'vceovr', 'roam', 'change_mou', 'change_rev', 'drop_vce', 'blck_vce', 'unan_vce', 'recv_vce', 'custcare', 'ccrndmou', 'mou_cvce', 'mou_rvce', 'owylis_vce', 'mouowylisv', 'iwylis_vce', 'mouiwylisv', 'mou_peav', 'opk_vce', 'mou_opkv', 'drop_blk', 'complete', 'callwait', 'months', 'uniqsubs', 'actvsubs', 'crclscod', 'adjrev', 'adjmou', 'avgrev', 'avgmou', 'avgqty', 'avg3qty', 'avg6rev', 'hnd_price', 'phones', 'models', 'lor', 'adults', 'income', 'numbcars', 'creditcd', 'eqpdays', 'new_cell_N', 'new_cell_U', 'new_cell_Y', 'prizm_social_one_C', 'prizm_social_one_R', 'prizm_social_one_S', 'prizm_social_one_T', 'prizm_social_one_U', 'area_ATLANTIC SOUTH', 'area_CALIFORNIA NORTH', 'area_CENTRAL/SOUTH TEXAS', 'area_CHICAGO', 'area_DALLAS', 'area_DC/MARYLAND/VIRGINIA', 'area_GREAT LAKES', 'area_HOUSTON', 'area_LOS ANGELES', 'area_MIDWEST', 'area_NEW ENGLAND', 'area_NEW YORK CITY', 'area_NORTH FLORIDA', 'area_NORTHWEST/ROCKY MOUNTAIN', 'area_OHIO', 'area

# Carga PREDICT dataset

In [ ]:
df_predict = pd.read_csv('/content/drive/MyDrive/Master_Data_Science/DSC0324BCN-ENTREGABLE_2-Barria_Bau_Guimerà_Maninatt_Solieri/telecom_churn_PREDICT.csv')

In [ ]:
df_predict.head()

,rev,mou,totmrc,da,ovrmou,ovrrev,vceovr,datovr,roam,change_mou,change_rev,drop_vce,drop_dat,blck_vce,blck_dat,unan_vce,unan_dat,plcd_vce,plcd_dat,recv_vce,recv_sms,comp_vce,comp_dat,custcare,ccrndmou,cc_mou,inonemin,threeway,mou_cvce,mou_cdat,mou_rvce,owylis_vce,mouowylisv,iwylis_vce,mouiwylisv,peak_vce,peak_dat,mou_peav,mou_pead,opk_vce,opk_dat,mou_opkv,mou_opkd,drop_blk,attempt,complete,callfwdv,callwait,months,uniqsubs,actvsubs,new_cell,crclscod,asl_flag,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,prizm_social_one,area,dualband,refurb_new,hnd_price,phones,models,hnd_webcap,truck,rv,ownrent,lor,dwlltype,marital,adults,infobase,income,numbcars,HHstatin,dwllsize,forgntvl,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays,Customer_ID
0,30.8350,136.75,29.9900,0.2475,1.25,0.500,0.5,0.000,0.0975,48.25,1.1550,2.000000,0.000000,10.333333,0.0,13.333333,0.0,66.000000,0.000000,3.333333,0.0,40.666667,0.000000,1.333333,1.333333,0.896667,0.666667,0.333333,71.746667,0.000000,5.406667,1.333333,0.713333,0.333333,0.473333,30.000000,0.0,54.500000,0.0,13.666667,0.000000,22.653333,0.000000,12.333333,66.000000,40.666667,0.0,0.000000,7,1,1,U,AA,N,393,958.0,248.46,182.94,958.0,393,30.49,159.67,65.50,121,52,30,160.0,66.0,31.0,C,ATLANTIC SOUTH AREA,Y,N,149.98999,1.0,1.0,WCMB,0.0,0.0,R,NaN,M,M,1.0,M,3.0,NaN,C,NaN,1.0,U,U,U,U,U,U,Y,216.0,1090001
1,35.8475,352.75,24.2700,0.4950,23.25,9.285,8.7,0.585,1.8000,-352.75,-35.8475,9.000000,0.000000,0.000000,0.0,10.000000,0.0,82.666667,1.000000,31.000000,0.0,65.333333,1.000000,3.666667,9.000000,5.890000,18.333333,1.333333,98.543333,1.023333,83.666667,15.666667,12.683333,1.333333,14.286667,38.000000,0.0,106.926667,0.0,43.666667,1.000000,75.283333,1.023333,9.000000,83.666667,66.333333,0.0,1.333333,7,1,1,U,EA,Y,1641,4011.0,391.87,305.59,3998.0,1634,50.93,666.33,272.33,470,181,48,666.0,272.0,51.0,C,NORTHWEST/ROCKY MOUNTAIN AREA,T,N,99.98999,2.0,1.0,WCMB,0.0,0.0,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,0.0,N,U,U,U,U,U,N,101.0,1090002
2,30.3275,241.50,39.9900,0.0000,0.00,0.000,0.0,0.000,0.0000,-86.50,0.8625,2.000000,0.000000,4.666667,0.0,20.666667,0.0,85.333333,0.000000,26.666667,0.0,59.333333,0.000000,4.333333,6.333333,4.466667,17.000000,1.666667,143.530000,0.000000,27.373333,9.666667,3.760000,19.666667,21.736667,48.666667,0.0,76.356667,0.0,36.000000,0.000000,94.543333,0.000000,6.666667,85.333333,59.333333,0.0,0.333333,9,2,2,Y,A,N,780,1889.0,312.91,219.45,1772.0,734,31.35,253.14,104.86,270,107,30,273.0,109.0,31.0,T,CENTRAL/SOUTH TEXAS AREA,N,N,129.98999,1.0,1.0,WCMB,1.0,0.0,O,13.0,S,M,3.0,M,6.0,1.0,C,A,0.0,N,U,U,U,U,U,Y,262.0,1090003
3,154.6925,2297.00,149.9900,4.7025,0.00,0.000,0.0,0.000,0.0000,119.00,0.2475,20.333333,0.000000,5.000000,0.0,329.333333,0.0,1067.000000,0.000000,628.000000,0.0,723.000000,0.000000,0.333333,0.666667,0.376667,408.000000,0.000000,769.136667,0.000000,679.020000,271.000000,167.090000,68.666667,150.656667,723.000000,0.0,961.736667,0.0,346.000000,0.000000,486.420000,0.000000,25.333333,1067.000000,723.000000,0.0,8.333333,11,1,1,U,AA,N,9018,18478.0,2217.05,2131.17,18478.0,9018,213.12,1847.80,901.80,2257,1045,155,2452.0,1171.0,278.0,S,ATLANTIC SOUTH AREA,N,R,79.98999,2.0,1.0,WCMB,0.0,0.0,O,3.0,S,U,3.0,M,6.0,1.0,NaN,A,0.0,S,U,U,U,U,U,Y,127.0,1090004
4,156.0050,542.25,48.9475,0.7425,70.75,28.295,28.1,0.195,0.0000,195.75,40.0150,5.666667,0.333333,0.666667,0.0,46.333333,0.0,167.666667,0.666667,11.333333,0.0,115.000000,0.333333,21.666667,48.000000,31.346667,10.000000,0.333333,343.066667,0.010000,11.243333,13.000000,14.563333,1.333333,2.100000,91.000000,0.0,205.216667,0.0,31.000000,0.333333,149.096667,0.010000,6.666667,168.333333,115.333333,0.0,0.666667,9,1,1,U,CA,Y,791,2601.0,879.51,608.66,2601.0,791,86.95,371.57,113.00,477,130,143,349.0,103.0,91.0,U,LOS ANGELES AREA,Y,N,149.98999,2.0,1.0,WCMB,0.0,0.0,O,1.0,NaN,S,1.0,M,6.0,1.0,C,NaN,0.0,N,U,U,U,U,U,Y,37.0,1090005


In [ ]:
df_predict.shape

(10000, 99)

In [ ]:
df_predict.duplicated().sum()

0

##### Importar dictionary data

In [ ]:
dictionary = pd.read_csv('/content/drive/MyDrive/Master_Data_Science/DSC0324BCN-ENTREGABLE_2-Barria_Bau_Guimerà_Maninatt_Solieri/variable_dictionary.csv')
columns_dict = dictionary.set_index('Variable')[' Description'].to_dict()
def descrip(columns):
    for col in columns:
        if col in columns_dict:
            print(f'{col}: {columns_dict[col]}')
        else:
            print(f'{col}: No hay descripción disponible')

print("Hacemos una funcion llamada 'descrip(columns)' para sacar la difinicion de las columnas de dataset")


Hacemos una funcion llamada 'descrip(columns)' para sacar la difinicion de las columnas de dataset


# ML Preprocessing

Se deben realizar las mismas transformaciones que se aplicaron en el preprocessing del train:
  * Eliminar mismas variables
  * Imputar mismos valores a nulos
  * Aplicar mismo encoding de categoricos
  * ...todo lo necesario para replicar las variables con las que el modelo se entrenó

NO SE DEBEN repetir:
  * Analisis visual de los datos
  * Analisis de correlaciones
  * Analisis de varianza
  * Calculo de metricas que varien con la distribución

##### Funciones

In [ ]:
deleted_vars = set() #definimos un set para ir guardando el registro de variables eliminadas
total_filas_eliminadas = 0
def deleted_columns(actual, anterior):
    global deleted_vars, total_filas_eliminadas  # Usamos las variables globales para mantener el seguimiento acumulado

    columnas_anteriores = set(anterior.columns)
    columnas_actuales = set(actual.columns)
    c_eliminadas = columnas_anteriores - columnas_actuales
    filas_eliminadas = len(anterior) - len(actual)
    total_filas_eliminadas += filas_eliminadas
    pct_filas_eliminadas = round(total_filas_eliminadas / len(anterior) * 100, 2)
    deleted_vars.update(c_eliminadas)  # Actualizamos el conjunto con las nuevas columnas eliminadas

    print(f"""Columnas eliminadas: {len(deleted_vars)}
	Pct de filas eliminadas acumuladas: {pct_filas_eliminadas}%""")

    descrip(c_eliminadas)

    return deleted_vars

print("Esta funcion me muestra las columnas que he eliminado")

Esta funcion me muestra las columnas que he eliminado


In [ ]:
def summarize_dataframe(df, dictionary):
    summary_data = { #creamos las listas
        'Column_Name': [],
        'Description': [],
        'Number_of_Nulls': [],
        'Pct_of_Nulls': [],
        'Mean': [],
        'Median': [],
        'Mode': []
    }

    total_rows = len(df)

    for column in df.columns: #iteramos sobre todo el dataset para sacar solos las columnas que tengan nulos
        null_count = df[column].isnull().sum()
        Pct_of_Nulls = (null_count / total_rows) * 100 #calculamos los porcentajes

        if null_count > 0: #Si tiene nulos, entonces comenzamos a llenar el diccionario
            summary_data['Column_Name'].append(column)
            summary_data['Description'].append(dictionary.get(column, 'No description available'))
            summary_data['Number_of_Nulls'].append(null_count)
            summary_data['Pct_of_Nulls'].append(Pct_of_Nulls)

            if df[column].dtype in ['int64', 'float64']:
                summary_data['Mean'].append(df[column].mean())
                summary_data['Median'].append(df[column].median())
                summary_data['Mode'].append(df[column].mode()[0] if not df[column].mode().empty else None)
            else:
                summary_data['Mean'].append(None)
                summary_data['Median'].append(None)
                summary_data['Mode'].append(df[column].mode()[0] if not df[column].mode().empty else None)


    summary_df = pd.DataFrame(summary_data)
    summary_df = summary_df.sort_values(by='Number_of_Nulls', ascending=False).reset_index(drop=True)
    return summary_df
print("Esta funcion crea un Dataframe con los nulos, su procentaje y su mean, median y mode")

Esta funcion crea un Dataframe con los nulos, su procentaje y su mean, median y mode


In [ ]:
print('''Eliminamos el Customer_ID del dataset y lo guardamos para poder después identificar los clientes
y asignarles su predicciones.''')
customer_id_list = df_predict.pop('Customer_ID')

Eliminamos el Customer_ID del dataset y lo guardamos para poder después identificar los clientes
y asignarles su predicciones.


In [ ]:
deleted_columns_list = joblib.load('/content/drive/MyDrive/Master_Data_Science/DSC0324BCN-ENTREGABLE_2-Barria_Bau_Guimerà_Maninatt_Solieri/Scaler/deleted_columns_list')
print('Cargamos la lista de las columnas que eliminamos durante el preprocessing del dataset de train')

Cargamos la lista de las columnas que eliminamos durante el preprocessing del dataset de train


In [ ]:
deleted_columns_list

['unan_dat',
 'ethnic_X',
 'avg3mou',
 'plcd_dat',
 'Customer_ID',
 'callfwdv',
 'hnd_webcap',
 'infobase',
 'recv_sms',
 'dwlltype',
 'opk_dat',
 'kid0_2',
 'totcalls',
 'totmou',
 'truck',
 'dwllsize',
 'avg6qty',
 'dualband',
 'kid11_15',
 'asl_flag',
 'ownrent',
 'adjqty',
 'comp_dat',
 'drop_dat',
 'peak_dat',
 'inonemin',
 'ovrrev',
 'threeway',
 'mou_pead',
 'forgntvl',
 'attempt',
 'ethnic_M',
 'peak_vce',
 'blck_dat',
 'mou_cdat',
 'refurb_new',
 'totrev',
 'kid16_17',
 'ovrmou',
 'plcd_vce',
 'rev',
 'avg6mou',
 'comp_vce',
 'kid6_10',
 'cc_mou',
 'rv',
 'ethnic_D',
 'mou_opkd',
 'avg3rev',
 'ethnic_P',
 'datovr',
 'kid3_5']

In [ ]:
len(deleted_columns_list)

52

In [ ]:
deleted = []
for col in df_predict.columns:
  if col in deleted_columns_list:
    df_predict.drop(col,axis=1,inplace=True)
    deleted.append(col)
print(f'Se han eliminado las siguientes columnas:\n {deleted}')

Se han eliminado las siguientes columnas:
 ['rev', 'ovrmou', 'ovrrev', 'datovr', 'drop_dat', 'blck_dat', 'unan_dat', 'plcd_vce', 'plcd_dat', 'recv_sms', 'comp_vce', 'comp_dat', 'cc_mou', 'inonemin', 'threeway', 'mou_cdat', 'peak_vce', 'peak_dat', 'mou_pead', 'opk_dat', 'mou_opkd', 'attempt', 'callfwdv', 'asl_flag', 'totcalls', 'totmou', 'totrev', 'adjqty', 'avg3mou', 'avg3rev', 'avg6mou', 'avg6qty', 'dualband', 'refurb_new', 'hnd_webcap', 'truck', 'rv', 'ownrent', 'dwlltype', 'infobase', 'dwllsize', 'forgntvl', 'kid0_2', 'kid3_5', 'kid6_10', 'kid11_15', 'kid16_17']


##### Imputación nulos

In [ ]:
summarize_dataframe(df_predict, columns_dict)
nulos_dataset = summarize_dataframe(df_predict, columns_dict)
nulos_dataset

,Column_Name,Description,Number_of_Nulls,Pct_of_Nulls,Mean,Median,Mode
0,numbcars,Known number of vehicles,5798,57.98,1.552832,2.00000,1.0
1,HHstatin,Premier household status indicator,4891,48.91,NaN,NaN,C
2,lor,Length of residence,3688,36.88,6.140843,5.00000,2.0
3,income,Estimated income,3240,32.40,5.520710,6.00000,6.0
4,adults,Number of adults in household,2986,29.86,2.451811,2.00000,1.0
5,avg6rev,Average monthly revenue over the previous six...,1432,14.32,62.994631,53.00000,35.0
6,prizm_social_one,Social group letter only,851,8.51,NaN,NaN,S
7,change_mou,Percentage change in monthly minutes of use v...,296,2.96,-15.286609,-4.25000,0.0
8,change_rev,Percentage change in monthly revenue vs previ...,296,2.96,-0.527241,-0.24750,0.0
9,creditcd,Credit card indicator,281,2.81,NaN,NaN,Y


Seguimos la misma estrategia usada en el preprocessing del dataset de train:
- numbcars ---> Imputado valor 0
- HHstatin, lor, income, adults, prizm_social_one ---> fillna con metodo pad.
- Columnas con menos del 5% de nulos ---> a diferencia del dataset de train, como aquí no podemos borrar los registros correspondientes, reemplazaremos los nulos con la moda.

Las modas atribuidas son las del propio dataset a predecir, dado que aqui lo que queremos es calcular la probabilidad de abandono de los clientes actuales para el proximo mes, nos parece mas apropriado usar los dados mas actualizados, para determinar las modas.

In [ ]:
df_predict['numbcars'].fillna(0.0,inplace=True)
print("Asumiendo que los nulos en numbcars significan que NO tienen coches, lo reemplazamos por 0")

Asumiendo que los nulos en numbcars significan que NO tienen coches, lo reemplazamos por 0


In [ ]:
df_predict['HHstatin'].fillna(method='pad', inplace=True)
print("Para mantener una distribucion real, hemos hecho un fillna con metodo pad")

Para mantener una distribucion real, hemos hecho un fillna con metodo pad


In [ ]:
df_predict['lor'].fillna(method='pad', inplace=True)
print("Para mantener una distribucion real hemos hecho un fillna con metodo pad")

Para mantener una distribucion real hemos hecho un fillna con metodo pad


In [ ]:
df_predict['income'].fillna(method='pad', inplace=True)
print("Para mantener una distribucion real hemos hecho un fillna con metodo pad")

Para mantener una distribucion real hemos hecho un fillna con metodo pad


In [ ]:
df_predict['adults'].fillna(method='pad', inplace=True)
print("Para mantener una distribucion real hemos hecho un fillna con metodo pad")

Para mantener una distribucion real hemos hecho un fillna con metodo pad


In [ ]:
df_predict['prizm_social_one'].fillna(method='pad', inplace=True)
print("Para mantener una distribucion real hemos hecho un fillna con metodo pad")

Para mantener una distribucion real hemos hecho un fillna con metodo pad


In [ ]:
print('Se ha imputado la moda de la columna a los nulos de todas las variables con menos del 5% de nulos.')
for col in summarize_dataframe(df_predict,columns_dict)['Column_Name']:
  df_predict[col].fillna(df_predict[col].mode()[0],inplace=True)

Se ha imputado la moda de la columna a los nulos de todas las variables con menos del 5% de nulos.


In [ ]:
df_predict.isnull().sum().sum()

0

##### Encoding categoricos

In [ ]:
df_predict.select_dtypes('O').head()

,new_cell,crclscod,prizm_social_one,area,marital,HHstatin,ethnic,creditcd
0,U,AA,C,ATLANTIC SOUTH AREA,M,C,U,Y
1,U,EA,C,NORTHWEST/ROCKY MOUNTAIN AREA,U,C,N,N
2,Y,A,T,CENTRAL/SOUTH TEXAS AREA,M,C,N,Y
3,U,AA,S,ATLANTIC SOUTH AREA,U,C,S,Y
4,U,CA,U,LOS ANGELES AREA,S,C,N,Y


##### creditcd - Indicador Boolean

In [ ]:
df_predict['creditcd'].value_counts()

creditcd
Y    5799
N    4201
Name: count, dtype: int64

In [ ]:
df_predict['creditcd'] = df_predict['creditcd'].map({'Y':1, 'N':0})
df_predict['creditcd'].value_counts()

creditcd
1    5799
0    4201
Name: count, dtype: int64

##### crclscod - Ordinal Encoding

In [ ]:
df_predict['crclscod'].value_counts()

crclscod
AA    2357
BA    1397
A     1394
CA     945
EA     898
E4     528
DA     468
B      367
ZA     316
D4     194
CY     169
Z4     159
ZY     107
D5      98
A2      90
B2      67
C5      67
U       51
Z5      48
C       38
GY      29
C2      28
J       21
Y       16
M       16
EC      16
GA      16
W       13
K       13
I       12
EM      11
U1      10
E        9
CC       6
G        5
Z        4
P1       4
D        3
JF       2
E2       2
L        2
H        1
Z1       1
Z2       1
O        1
Name: count, dtype: int64

In [ ]:
df_predict['crclscod']=df_predict['crclscod'].map(lambda x:x[0])
print('En la columna "crclscod" se han agrupado las clases en función de la primera letra de cada clase. Ejemplo: A,AA,A2,A3 ---> A')

En la columna "crclscod" se han agrupado las clases en función de la primera letra de cada clase. Ejemplo: A,AA,A2,A3 ---> A


In [ ]:
df_predict['crclscod'].value_counts()

crclscod
A    3841
B    1831
E    1464
C    1253
D     763
Z     636
U      61
G      50
J      23
M      16
Y      16
W      13
K      13
I      12
P       4
L       2
H       1
O       1
Name: count, dtype: int64

In [ ]:
crclscod_sorted_values = list(df_predict['crclscod'].unique())
crclscod_sorted_values.sort()
print(crclscod_sorted_values)

['A', 'B', 'C', 'D', 'E', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'O', 'P', 'U', 'W', 'Y', 'Z']


In [ ]:
oe_crclscod = OrdinalEncoder(categories=[crclscod_sorted_values])
oe_crclscod

OrdinalEncoder(categories=[['A', 'B', 'C', 'D', 'E', 'G', 'H', 'I', 'J', 'K',
                            'L', 'M', 'O', 'P', 'U', 'W', 'Y', 'Z']])

In [ ]:
df_predict["crclscod"]=oe_crclscod.fit_transform(df_predict[['crclscod']])

In [ ]:
df_predict['crclscod'].value_counts()

crclscod
0.0     3841
1.0     1831
4.0     1464
2.0     1253
3.0      763
17.0     636
14.0      61
5.0       50
8.0       23
11.0      16
16.0      16
15.0      13
9.0       13
7.0       12
13.0       4
10.0       2
6.0        1
12.0       1
Name: count, dtype: int64

##### area - Limpieza formato

In [ ]:
df_predict.select_dtypes('O')

,new_cell,prizm_social_one,area,marital,HHstatin,ethnic
0,U,C,ATLANTIC SOUTH AREA,M,C,U
1,U,C,NORTHWEST/ROCKY MOUNTAIN AREA,U,C,N
2,Y,T,CENTRAL/SOUTH TEXAS AREA,M,C,N
3,U,S,ATLANTIC SOUTH AREA,U,C,S
4,U,U,LOS ANGELES AREA,S,C,N
...,...,...,...,...,...,...
9995,U,U,LOS ANGELES AREA,M,C,S
9996,U,S,LOS ANGELES AREA,M,I,N
9997,U,U,SOUTHWEST AREA,B,I,U
9998,U,U,SOUTHWEST AREA,U,I,S


In [ ]:
df_predict['area']

0                 ATLANTIC SOUTH AREA
1       NORTHWEST/ROCKY MOUNTAIN AREA
2            CENTRAL/SOUTH TEXAS AREA
3                 ATLANTIC SOUTH AREA
4                    LOS ANGELES AREA
                    ...              
9995                 LOS ANGELES AREA
9996                 LOS ANGELES AREA
9997                   SOUTHWEST AREA
9998                   SOUTHWEST AREA
9999                 LOS ANGELES AREA
Name: area, Length: 10000, dtype: object

In [ ]:
df_predict['area'] = df_predict['area'].str.strip().str.replace(' AREA', '')
print('Eliminamos la palabra "area" en cada observación de la columna "area" porque es redundante')

Eliminamos la palabra "area" en cada observación de la columna "area" porque es redundante


In [ ]:
df_predict['area']

0                 ATLANTIC SOUTH
1       NORTHWEST/ROCKY MOUNTAIN
2            CENTRAL/SOUTH TEXAS
3                 ATLANTIC SOUTH
4                    LOS ANGELES
                  ...           
9995                 LOS ANGELES
9996                 LOS ANGELES
9997                   SOUTHWEST
9998                   SOUTHWEST
9999                 LOS ANGELES
Name: area, Length: 10000, dtype: object

##### One Hot Enconding

In [ ]:
df_predict = pd.get_dummies(df_predict,dtype=int)
print("Hacemos el One Hot Encoding en las variables tipo Object que quedan")

Hacemos el One Hot Encoding en las variables tipo Object que quedan


In [ ]:
df_predict.shape

(10000, 100)

In [ ]:
df_predict

,mou,totmrc,da,vceovr,roam,change_mou,change_rev,drop_vce,blck_vce,unan_vce,recv_vce,custcare,ccrndmou,mou_cvce,mou_rvce,owylis_vce,mouowylisv,iwylis_vce,mouiwylisv,mou_peav,opk_vce,mou_opkv,drop_blk,complete,callwait,months,uniqsubs,actvsubs,crclscod,adjrev,adjmou,avgrev,avgmou,avgqty,avg3qty,avg6rev,hnd_price,phones,models,lor,adults,income,numbcars,creditcd,eqpdays,new_cell_N,new_cell_U,new_cell_Y,prizm_social_one_C,prizm_social_one_R,prizm_social_one_S,prizm_social_one_T,prizm_social_one_U,area_ATLANTIC SOUTH,area_CALIFORNIA NORTH,area_CENTRAL/SOUTH TEXAS,area_CHICAGO,area_DALLAS,area_DC/MARYLAND/VIRGINIA,area_GREAT LAKES,area_HOUSTON,area_LOS ANGELES,area_MIDWEST,area_NEW ENGLAND,area_NEW YORK CITY,area_NORTH FLORIDA,area_NORTHWEST/ROCKY MOUNTAIN,area_OHIO,area_PHILADELPHIA,area_SOUTH FLORIDA,area_SOUTHWEST,area_TENNESSEE,marital_A,marital_B,marital_M,marital_S,marital_U,HHstatin_A,HHstatin_B,HHstatin_C,HHstatin_G,HHstatin_H,HHstatin_I,ethnic_B,ethnic_C,ethnic_D,ethnic_F,ethnic_G,ethnic_H,ethnic_I,ethnic_J,ethnic_M,ethnic_N,ethnic_O,ethnic_P,ethnic_R,ethnic_S,ethnic_U,ethnic_X,ethnic_Z
0,136.75,29.9900,0.2475,0.5,0.0975,48.25,1.1550,2.000000,10.333333,13.333333,3.333333,1.333333,1.333333,71.746667,5.406667,1.333333,0.713333,0.333333,0.473333,54.500000,13.666667,22.653333,12.333333,40.666667,0.000000,7,1,1,0.0,182.94,958.0,30.49,159.67,65.50,52,31.0,149.98999,1.0,1.0,2.0,1.0,3.0,0.0,1,216.0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,352.75,24.2700,0.4950,8.7,1.8000,-352.75,-35.8475,9.000000,0.000000,10.000000,31.000000,3.666667,9.000000,98.543333,83.666667,15.666667,12.683333,1.333333,14.286667,106.926667,43.666667,75.283333,9.000000,66.333333,1.333333,7,1,1,4.0,305.59,3998.0,50.93,666.33,272.33,181,51.0,99.98999,2.0,1.0,2.0,1.0,3.0,0.0,0,101.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,241.50,39.9900,0.0000,0.0,0.0000,-86.50,0.8625,2.000000,4.666667,20.666667,26.666667,4.333333,6.333333,143.530000,27.373333,9.666667,3.760000,19.666667,21.736667,76.356667,36.000000,94.543333,6.666667,59.333333,0.333333,9,2,2,0.0,219.45,1772.0,31.35,253.14,104.86,107,31.0,129.98999,1.0,1.0,13.0,3.0,6.0,1.0,1,262.0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,2297.00,149.9900,4.7025,0.0,0.0000,119.00,0.2475,20.333333,5.000000,329.333333,628.000000,0.333333,0.666667,769.136667,679.020000,271.000000,167.090000,68.666667,150.656667,961.736667,346.000000,486.420000,25.333333,723.000000,8.333333,11,1,1,0.0,2131.17,18478.0,213.12,1847.80,901.80,1045,278.0,79.98999,2.0,1.0,3.0,3.0,6.0,1.0,1,127.0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,542.25,48.9475,0.7425,28.1,0.0000,195.75,40.0150,5.666667,0.666667,46.333333,11.333333,21.666667,48.000000,343.066667,11.243333,13.000000,14.563333,1.333333,2.100000,205.216667,31.000000,149.096667,6.666667,115.333333,0.666667,9,1,1,2.0,608.66,2601.0,86.95,371.57,113.00,130,91.0,149.98999,2.0,1.0,1.0,1.0,6.0,1.0,1,37.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.00,44.9900,0.0000,0.0,0.0000,0.00,0.0000,2.000000,0.000000,7.666667,3.000000,0.000000,0.000000,94.866667,2.016667,2.666667,1.680000,0.000000,0.000000,23.566667,26.666667,73.316667,2.000000,45.333333,0.000000,25,1,1,1.0,1561.14,9494.0,74.34,452.10,178.62,121,100.0,29.98999,1.0,1.0,15.0,5.0,4.0,2.0,1,773.0,0,1,0,0,0,0,0,1

# Check model features

* Comprobar que tenemos en el dataset preprocesado todas las model features, de lo contrario no podremos hacer predict.
* Ordenar las variables en mismo orden que las model features

In [ ]:
# verificamos que los dataset tienen el mismo número de variables

df_predict.shape[1] == len(feature_names)

False

In [ ]:
print(f'Numero de columnas dataseta predecir: {df_predict.shape[1]}')
print(f'Numero de features del modelo: {len(feature_names)}')

Numero de columnas dataseta predecir: 100
Numero de features del modelo: 95


In [ ]:
# columnas que están en el dataset de predicción pero no en el dataset de entrenamiento (nueva categoría)

col_sobrantes = []
for col in df_predict.columns:
  if col not in feature_names:
    col_sobrantes.append(col)
col_sobrantes

['ethnic_C', 'ethnic_D', 'ethnic_M', 'ethnic_P', 'ethnic_X']

In [ ]:
# eliminamos las columnas sobrantes, ya que el modelo no ha sido entrenado con ellas

df_predict.drop(col_sobrantes, axis=1, inplace=True)
print(f'Se han eliminado las siguientes columnas: {col_sobrantes}')

Se han eliminado las siguientes columnas: ['ethnic_C', 'ethnic_D', 'ethnic_M', 'ethnic_P', 'ethnic_X']


In [ ]:
# columnas que están en el dataset de entrenamiento pero no están en el dataset de predicción
col_faltantes = []
for col in feature_names:
  if col not in df_predict.columns:
    col_faltantes.append(col)
col_faltantes
print(f'Columnas faltantes en el dataset a predecir: {col_faltantes}')

Columnas faltantes en el dataset a predecir: []


In [ ]:
# verificamos que los dataset tienen el mismo número de variables
df_predict.shape[1] == len(feature_names)

True

In [ ]:
# ponemos las columnas del dataset de predicción en el mismo orden que el de entrenamiento
columnas = feature_names
df_predict = df_predict.reindex(columns=columnas)

# Rescaling

* Si se entrenó el modelo con un dataset estandarizado, estandarizar con mismo scaler.

# PREDICT

* predict() y predict_proba()

In [ ]:
y_pred = cboost.predict(df_predict)
y_pred_proba = cboost.predict_proba(df_predict)[:,1]

predictions = pd.DataFrame({'CUSTOMER_ID':customer_id_list,
                            'PREDICT':y_pred,
                            'PREDICT_PROBA':y_pred_proba})

In [ ]:
predictions.set_index('CUSTOMER_ID',inplace=True)

# Guarda predicciones

* Guardar las predicciones en data path. Cada fila debe estar etiquetada con el ID.

In [ ]:
predictions

,PREDICT,PREDICT_PROBA
CUSTOMER_ID,,
1090001,0,0.354265
1090002,0,0.380376
1090003,0,0.145326
1090004,0,0.394430
1090005,0,0.296930
...,...,...
1099996,1,0.720647
1099997,1,0.721583
1099998,1,0.794220


In [ ]:
predictions.to_csv('/content/drive/MyDrive/Master_Data_Science/DSC0324BCN-ENTREGABLE_2-Barria_Bau_Guimerà_Maninatt_Solieri/Predict/churn_predictions.csv')